In [1]:
# Importing some libraries
import graphlab as gl
import numpy as np
import pandas as pd
import pickle
import os
from  sklearn.preprocessing import MinMaxScaler
import warnings
## Ignoring Warnings
warnings.filterwarnings('ignore')
# Defining Path for raw files
#path = "C:/Users/Aayush - Carlson/Desktop/PA/"

## 1) Data import

In [2]:
# Import Files
data = pd.read_csv('ratings.csv')
data.columns = ['user_id','item_id','rating','time']
users = pd.read_csv('users.csv')
movies = pickle.load(open('movies.p'))
movies.columns = ['item_id','title','genre','plot']
movies_omdb = pd.read_csv('movie_omdb.csv')
movies_expanded = pd.read_csv('movies_expanded.csv').ix[:,1:]
x = list(movies_expanded.columns)
x[0] = 'item_id'
movies_expanded.columns = x
movies_expanded = movies_expanded.drop(['movie_title','genre'],axis=1)
movies_name = gl.SFrame('C:/Users/Aayush - Carlson/Desktop/PA/movies.csv')
movies_name = gl.SFrame({'item_id': movies_name['movie_id'],'title' : movies_name['movie_title'],'genre': movies_name['genre']})

## 2) Data Manipulation

In [3]:
### Defining all the fucntions for Manipulations

# Function to Removing "tt" from the movie_id
def subtring(x):
    return (int(x[2:]))

## Function for Extracting numerical run time from text
def runtime(x):
    if pd.isnull(x):
        return np.NaN
    elif len(x.split()) == 2:
        return int(x.split()[0])
    elif len(x.split()) == 4:
        return int(x.split()[0]) * 60 + int(x.split()[2])
    
## Function if a movie won a award or not
def award(x):
    if pd.isnull(x):
        return 0
    else:
        return 1

## Function if the item is a Movie or not, assuming Nulls are also movies
def is_movie(x):
    if pd.isnull(x):
        return 1
    elif x == "movie":
        return 1
    else:
        return 0

In [4]:
## Fixing movie_id in omdb table - Removing tt from string and converting into a integer
movies_omdb['item_id'] = movies_omdb.imdbID.apply(subtring)

#Implementing the above function for data transformation
movies_omdb.Runtime = movies_omdb.Runtime.apply(runtime)

# Taking only selective columns from OMDB movie data
movies_omdb1 = movies_omdb.ix[:,[1,12,15,16,18,20]] 

# Applying remaining transformation
movies_omdb1.Awards = movies_omdb1.Awards.apply(award)
movies_omdb1.Type = movies_omdb1.Type.apply(is_movie)
movies_omdb1.imdbRating[pd.isnull(movies_omdb1.imdbRating)] =  np.mean(movies_omdb1.imdbRating )
movies_omdb1.imdbVotes[pd.isnull(movies_omdb1.imdbVotes)] =  np.mean(movies_omdb1.imdbVotes )

# Merging OMDB features with Genres
movies_full = pd.merge(movies_expanded,movies_omdb1,how="inner")

In [5]:
## Converting data in SFrame
ratings = gl.SFrame(data.ix[:,[0,1,2]])
genre = gl.SFrame(movies_expanded)
omdb = gl.SFrame(movies_omdb1)
movie_features_full = gl.SFrame(movies_full)

[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: C:\Users\AAYUSH~1\AppData\Local\Temp\graphlab_server_1480798848.log.0


This non-commercial license of GraphLab Create for academic use is assigned to agraw134@umn.edu and will expire on November 11, 2017.


## 3) Modelling

In [6]:
# Validation Code for different algorithms together

# Creating Folds for Cross Validation
folds = gl.cross_validation.KFold(ratings, 5)

# List creating for saving Cross Validation Results
result_mf_rating = list()
result_rmf_rating = list()
result_mf_rating_genre = list()
result_mf_rating_omdb = list()
result_mf_rating_full = list()

# Running Models with Cross Validation
for train, valid in folds:
    # Model 1 : Factorization Machines with Rating Database
    mf_rating = gl.factorization_recommender.create(train, target='rating')
    
    # Model 2: Rank based Factorization Machines with Rating Database
    rmf_rating = gl.recommender.create(train, target='rating')
    
    # Model 3:  Factorization Machines with Rating Database and Movie Genre as side features
    mf_rating_genre = gl.factorization_recommender.create(train, target='rating',item_data = genre,side_data_factorization=True)
    
    # Model 4:  Factorization Machines with Rating Database and Movie OMDB as side features
    mf_rating_omdb = gl.factorization_recommender.create(train, target='rating',item_data = omdb,side_data_factorization=True)
    
    # Model 5:  Factorization Machines with Rating Database, Genre and  Movie OMDB as side features
    mf_rating_full = gl.factorization_recommender.create(train, target='rating',item_data = movie_features_full,side_data_factorization=True)
    
    # Evaluating Model 1 on Test set
    x = mf_rating.evaluate(valid)
    result_mf_rating.append(x['rmse_overall'])
    
    # Evaluating Model 2 on Test set
    x = rmf_rating.evaluate(valid)
    result_rmf_rating.append(x['rmse_overall'])
    
    # Evaluating Model 3 on Test set
    x = mf_rating_genre.evaluate(valid)
    result_mf_rating_genre.append(x['rmse_overall'])        
    
    # Evaluating Model 4 on Test set
    x = mf_rating_omdb.evaluate(valid)
    result_mf_rating_omdb.append(x['rmse_overall'])
    
    # Evaluating Model 5 on Test set
    x = mf_rating_full.evaluate(valid)
    result_mf_rating_full.append(x['rmse_overall'])

Recsys training: model = factorization_recommender

Preparing data set.

Data has 433200 observations with 36335 users and 23383 items.

Data prepared in: 0.27222s

Training factorization_recommender for recommendations.

...

recommendations finished on 6000/9584 queries. users per second: 190.066

recommendations finished on 7000/9584 queries. users per second: 190.268

recommendations finished on 8000/9584 queries. users per second: 188.333

recommendations finished on 9000/9584 queries. users per second: 183.878


Precision and recall summary statistics by cutoff
+--------+-------------------+-------------------+
| cutoff |   mean_precision  |    mean_recall    |
+--------+-------------------+-------------------+
|   1    |        0.0        |        0.0        |
|   2    |        0.0        |        0.0        |
|   3    | 3.47801892042e-05 | 7.14661422005e-07 |
|   4    | 2.60851419032e-05 | 7.14661422005e-07 |
|   5    | 4.17362270451e-05 | 0.000105055229035 |
|   6    | 5.21702838063e-05 | 0.000107801033446 |
|   7    |  5.9623181493e-05 | 0.000109028569535 |
|   8    | 5.21702838063e-05 | 0.000109028569535 |
|   9    | 4.63735856056e-05 | 0.000109028569535 |
|   10   | 4.17362270451e-05 | 0.000109028569535 |
+--------+-------------------+-------------------+
[10 rows x 3 columns]

('\nOverall RMSE: ', 1.6416101507963237)

Per User RMSE (best)
+---------+-------+-------------------+
| user_id | count |        rmse       |
+---------+-------+-------------------+
|  37739  |   1   | 0.0004038

In [10]:
## Printing the result of 5 fold Cross - Validation result
print "Model 1 : Factorization Machines with Rating Database :{0:.4f}".format(np.mean(result_mf_rating))
print "Model 2 : Rank based Factorization Machines with Rating Database :{0:.4f}".format(np.mean(result_rmf_rating))
print "Model 3 : Factorization Machines with Rating Database and Movie Genre as side features :{0:.4f}".format(np.mean(result_mf_rating_genre))
print "Model 4 : Factorization Machines with Rating Database and Movie OMDB as side features :{0:.4f}".format(np.mean(result_mf_rating_omdb))
print "Model 5 : Factorization Machines with Rating Database, Genre and  Movie OMDB as side features :{0:.4f}".format(np.mean(result_mf_rating_full))

Model 1 : Factorization Machines with Rating Database :1.6944
Model 2 : Rank based Factorization Machines with Rating Database :1.8760
Model 3 : Factorization Machines with Rating Database and Movie Genre as side features :1.8224
Model 4 : Factorization Machines with Rating Database and Movie OMDB as side features :1.6567
Model 5 : Factorization Machines with Rating Database, Genre and  Movie OMDB as side features :1.6498


## 4) Visualization

In [20]:
# As seen above Movie and Genre Features combined give better performance let's build one visualization on it
mf_rating_full = gl.ranking_factorization_recommender.create(ratings, target='rating',item_data = movie_features_full,side_data_factorization=True,random_seed=30)
view = mf_rating_full.views.explore(item_data=movies_name,item_name_column='title')
view.show()

Recsys training: model = ranking_factorization_recommender

Preparing data set.

Data has 541501 observations with 45339 users and 26096 items.

Data prepared in: 0.450346s

Training ranking_factorization_recommender for recommendations.

...

Optimization Complete: Maximum number of passes through the data reached.

Computing final objective value and training RMSE.

Final objective value: 1.56801

Final training RMSE: 0.87422

View object

URI: 		http://localhost:32212/view/2baa26f3-230b-4b5c-8612-b9f70569d53e
HTML: 		
<gl-recommender-explore
    uri="http://localhost:32212/view/6a0c5dd3-459a-4547-ba4f-8cabdfaa188d"
    api_key=""
/>
        